In [57]:
import torch
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

iris_file = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df = pd.read_csv(iris_file, sep=",",header=None,)
mapping = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2}
df = df.replace("Iris-versicolor",1)
df = df.replace("Iris-setosa",0)
df = df.replace("Iris-virginica",2)
print(df[0:2])
#columns are: 
#1. sepal length in cm 
#2. sepal width in cm 
#3. petal length in cm 
#4. petal width in cm 
#5. class: 
#-- Iris Setosa 
#-- Iris Versicolour 
#-- Iris Virginica
df_X = df.drop(4,axis=1)
df_X.head()
df_Y = df.drop([0,1,2,3],axis=1)
df_Y.head()
X_train,x_test,Y_train,y_test = train_test_split(df_X.values, df_Y.values, test_size=0.20, random_state=42)
print(X_train.shape,Y_train.shape,x_test.shape,y_test.shape)
#logisticregression wants (30,) w/no column. Flattened array
Y_train = np.ravel(Y_train)
y_test = np.ravel(y_test)
print(Y_train.shape,y_test.shape)

     0    1    2    3  4
0  5.1  3.5  1.4  0.2  0
1  4.9  3.0  1.4  0.2  0
(120, 4) (120, 1) (30, 4) (30, 1)
(120,) (30,)


In [54]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, solver='lbfgs',multi_class='multinomial').fit(X_train, np.ravel(Y_train))


In [67]:
print(x_test[0].reshape(-1,1).T)

whoami = clf.predict(x_test[0].reshape(-1,1).T)
print(whoami) #this is a Iris-versicolor. 
print(y_test[0])
#looks correct

[[6.1 2.8 4.7 1.2]]
[1]
1
